In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to cluster and characterize complaints

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

The goal of this notebook is to demonstrate a comment characterization algorithm for an online business. We will accomplish this using [Google's PaLM 2](https://ai.google/discover/palm2/) and [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) in three steps:

1. Use PaLM2TextEmbeddingGenerator to [generate text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) for each of 10000 complaints sent to an online bank. If you're not familiar with what a text embedding is, it's a list of numbers that are like coordinates in an imaginary "meaning space" for sentences. (It's like [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), but for more general text.) The important point for our purposes is that similar sentences are close to each other in this imaginary space.
2. Use KMeans clustering to group together complaints whose text embeddings are near to eachother. This will give us sets of similar complaints, but we don't yet know _why_ these complaints are similar.
3. Prompt GeminiTextGenerator in English asking what the difference is between the groups of complaints that we got. Thanks to the power of modern LLMs, the response might give us a very good idea of what these complaints are all about, but remember to ["understand the limits of your dataset and model."](https://ai.google/responsibility/responsible-ai-practices/#:~:text=Understand%20the%20limitations%20of%20your%20dataset%20and%20model)

We will tie these pieces together in Python using BigQuery DataFrames. [Click here](https://cloud.google.com/bigquery/docs/dataframes-quickstart) to learn more about BigQuery DataFrames!

### Dataset

This notebook uses the [CFPB Consumer Complaint Database](https://console.cloud.google.com/marketplace/product/cfpb/complaint-database).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)
* BigQuery ML
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models), [Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,aiplatform.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API
  * Vertex AI API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# set your project ID below
PROJECT_ID = "bigframes-dev"  # @param {type:"string"}

# Set the project id in gcloud
#! gcloud config set project {PROJECT_ID}

#### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [3]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [4]:
# from google.colab import auth
# auth.authenticate_user()

Now we are ready to use BigQuery DataFrames!

## Step 1: Text embedding 

BigQuery DataFrames setup

In [5]:
import bigframes.pandas as bf

# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bf.options.bigquery.project = PROJECT_ID

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bf.close_session()`. After that, you can reuse `bf.options.bigquery.location` to specify another location.

Data Input - read the data from a publicly available BigQuery dataset

In [6]:
input_df = bf.read_gbq("bigquery-public-data.cfpb_complaints.complaint_database")

In [7]:
issues_df = input_df[["consumer_complaint_narrative"]].dropna()
issues_df.peek(n=5) # View an arbitrary five complaints

,consumer_complaint_narrative
954305,HRRG is a debt collection company who has been...
954501,Someone open a XXXX account in my name that li...
954643,XXXX and capital one data breach caused my cre...
954923,Received call with local area code : XXXX. Rec...
955092,I have tried several times to resolve this acc...


Downsample DataFrame to 10,000 records for model training.

In [8]:
# Choose 10,000 complaints randomly and store them in a column in a DataFrame
downsampled_issues_df = issues_df.sample(n=10000)

Generate the text embeddings

In [9]:
from bigframes.ml.llm import PaLM2TextEmbeddingGenerator

model = PaLM2TextEmbeddingGenerator() # No connection id needed

In [10]:
# Will take ~3 minutes to compute the embeddings
predicted_embeddings = model.predict(downsampled_issues_df)
# Notice the lists of numbers that are our text embeddings for each complaint
predicted_embeddings

,text_embedding,statistics,ml_embed_text_status,content
300,[ 0.01977486 -0.04289974 -0.05289588 -0.027267...,"{""token_count"":498,""truncated"":false}",,XXXX XXXX XXXXXXXX has reported on my credit r...
566,[ 2.82494519e-02 -4.81299721e-02 -4.91684638e-...,"{""token_count"":51,""truncated"":false}",,My credit profile is inaccurate which is not f...
772,[ 3.19108889e-02 -7.79124349e-03 -6.68819547e-...,"{""token_count"":398,""truncated"":false}",,XXXX XXXX holds our lot loan that matured in X...
981,[ 4.21869569e-03 -1.31362844e-02 -2.08469853e-...,"{""token_count"":202,""truncated"":false}",,My Simple account was transferred to a BBVA a...
1103,[ 4.16752733e-02 -4.29417416e-02 -6.27184287e-...,"{""token_count"":31,""truncated"":false}",,The debt occurred more than 7 years in the pas...
1144,[ 2.73083095e-02 -4.39260900e-02 -5.70089184e-...,"{""token_count"":72,""truncated"":false}",,I'm really not sure what happened. I have mail...
1163,[ 1.32309105e-02 -1.49403308e-02 -3.24638188e-...,"{""token_count"":138,""truncated"":false}",,"I was about to graduate in XX/XX/2018, when I ..."
1610,[ 4.18850686e-03 -3.26682590e-02 -8.01362470e-...,"{""token_count"":371,""truncated"":false}",,I have disputed the referenced accounts to the...
1613,[ 5.40047046e-03 -5.22610508e-02 -1.18300663e-...,"{""token_count"":12,""truncated"":false}",,Disputed with credit beareu serveral times
1730,[ 2.67208274e-02 -1.20688053e-02 -5.07430844e-...,"{""token_count"":250,""truncated"":false}",,They attempted to collect an amount that was n...


The model may have encountered errors while calculating embeddings for some rows. Filter out the errored rows before training the model. Alternatively, select these rows and retry the embeddings.

In [12]:
successful_rows = predicted_embeddings["ml_embed_text_status"] == ""
predicted_embeddings = predicted_embeddings[successful_rows]
predicted_embeddings


,text_embedding,statistics,ml_embed_text_status,content
300,[ 0.01977486 -0.04289974 -0.05289588 -0.027267...,"{""token_count"":498,""truncated"":false}",,XXXX XXXX XXXXXXXX has reported on my credit r...
566,[ 2.82494519e-02 -4.81299721e-02 -4.91684638e-...,"{""token_count"":51,""truncated"":false}",,My credit profile is inaccurate which is not f...
772,[ 3.19108889e-02 -7.79124349e-03 -6.68819547e-...,"{""token_count"":398,""truncated"":false}",,XXXX XXXX holds our lot loan that matured in X...
981,[ 4.21869569e-03 -1.31362844e-02 -2.08469853e-...,"{""token_count"":202,""truncated"":false}",,My Simple account was transferred to a BBVA a...
1103,[ 4.16752733e-02 -4.29417416e-02 -6.27184287e-...,"{""token_count"":31,""truncated"":false}",,The debt occurred more than 7 years in the pas...
1144,[ 2.73083095e-02 -4.39260900e-02 -5.70089184e-...,"{""token_count"":72,""truncated"":false}",,I'm really not sure what happened. I have mail...
1163,[ 1.32309105e-02 -1.49403308e-02 -3.24638188e-...,"{""token_count"":138,""truncated"":false}",,"I was about to graduate in XX/XX/2018, when I ..."
1610,[ 4.18850686e-03 -3.26682590e-02 -8.01362470e-...,"{""token_count"":371,""truncated"":false}",,I have disputed the referenced accounts to the...
1613,[ 5.40047046e-03 -5.22610508e-02 -1.18300663e-...,"{""token_count"":12,""truncated"":false}",,Disputed with credit beareu serveral times
1730,[ 2.67208274e-02 -1.20688053e-02 -5.07430844e-...,"{""token_count"":250,""truncated"":false}",,They attempted to collect an amount that was n...


We now have the complaints and their text embeddings as two columns in our predicted_embeddings DataFrame.

## Step 2: Create k-means model and predict clusters

In [13]:
from bigframes.ml.cluster import KMeans

cluster_model = KMeans(n_clusters=10) # We will divide our complaints into 10 groups

Perform KMeans clustering

In [15]:
# Use KMeans clustering to calculate our groups. Will take ~3 minutes.
cluster_model.fit(predicted_embeddings[["text_embedding"]])
clustered_result = cluster_model.predict(predicted_embeddings)
# Notice the CENTROID_ID column, which is the ID number of the group that
# each complaint belongs to.
clustered_result.peek(n=5)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,text_embedding,statistics,ml_embed_text_status,content
3431779,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.535594290569...",[ 3.92934196e-02 -9.10845026e-02 -4.04280126e-...,"{""token_count"":10,""truncated"":false}",,Fraud scratch account in Experian credit report
41916,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.546894006330...",[-3.28768999e-03 -4.22948077e-02 -7.46203661e-...,"{""token_count"":10,""truncated"":false}",,I did not authorize this inquiry on my credit ...
308379,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.565107956565...",[ 4.27730894e-03 -6.67066649e-02 -7.86168799e-...,"{""token_count"":10,""truncated"":false}",,My parents credit information is on my credit ...
554444,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.399670811064...",[ 9.78547148e-03 -3.19877677e-02 -6.17879257e-...,"{""token_count"":100,""truncated"":false}",,I reviewed my credit report and it showed mult...
2045082,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.406139421970...",[ 0.00498423 -0.04450029 -0.05945096 -0.052494...,"{""token_count"":100,""truncated"":false}",,What is wrong why these accounts are on my cre...


Our DataFrame clustered_result now has an additional column that includes an ID from 1-10 (inclusive) indicating which semantically similar group they belong to.

## Step 3: Use Gemini to summarize complaint clusters

Build prompts - we will choose just two of our categories and prompt GeminiTextGenerator to identify their salient characteristics. The prompt is natural language in a python string.

In [16]:
# Using bigframes, with syntax identical to pandas,
# filter out the first and second groups
cluster_1_result = clustered_result[
    clustered_result["CENTROID_ID"] == 1
][["content"]]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[
    clustered_result["CENTROID_ID"] == 2
][["content"]]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

In [17]:
# Build plain-text prompts to send to PaLM 2. Use only 5 complaints from each group.
prompt1 = 'comment list 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas["content"].iloc[i] + '\n'

prompt2 = 'comment list 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
        cluster_2_result_pandas["content"].iloc[i] + '\n'

print(prompt1)
print(prompt2)

comment list 1:
1. The debt occurred more than 7 years in the past. It is illegal to report credit items after 7 years. There was no way to verify the debt.
2. I'm really not sure what happened. I have mailed off letters to the credit bureaus continuously and thus far I have not gotten a response. My name is XXXX XXXX and I am filing this complaint for falsely reporting misleading information. There is no third party involved. Please review the uploaded letters.
3. Fraudulent account removed from my credit report due to Identity Theft
4. This account was not mine I explained this and they closed it but are still reporting a balance with the fraudulent charges
5. Ive disputed a few times with Experian that I paid off my credit card account. The first time in XX/XX/2022 that they updated my account that it showed I had a {$0.00} balance remaining but wasnt showing paid!. Then the second dispute they just deleted the account on XX/XX/2022 causing my credit score to drop. I paid off a tota

In [18]:
# The plain English request we will make of PaLM 2
prompt = (
    "Please highlight the most obvious difference between "
    "the two lists of comments:\n" + prompt1 + prompt2
)
print(prompt)

Please highlight the most obvious difference between the two lists of comments:
comment list 1:
1. The debt occurred more than 7 years in the past. It is illegal to report credit items after 7 years. There was no way to verify the debt.
2. I'm really not sure what happened. I have mailed off letters to the credit bureaus continuously and thus far I have not gotten a response. My name is XXXX XXXX and I am filing this complaint for falsely reporting misleading information. There is no third party involved. Please review the uploaded letters.
3. Fraudulent account removed from my credit report due to Identity Theft
4. This account was not mine I explained this and they closed it but are still reporting a balance with the fraudulent charges
5. Ive disputed a few times with Experian that I paid off my credit card account. The first time in XX/XX/2022 that they updated my account that it showed I had a {$0.00} balance remaining but wasnt showing paid!. Then the second dispute they just dele

Get a response from Gemini by making a call to Vertex AI using our connection.

In [20]:
from bigframes.ml.llm import GeminiTextGenerator

q_a_model = GeminiTextGenerator()

In [21]:
# Make a DataFrame containing only a single row with our prompt for Gemini
df = bf.DataFrame({"prompt": [prompt]})

In [22]:
# Send the request for Gemini to generate a response to our prompt
major_difference = q_a_model.predict(df)
# PaLM 2's response is the only row in the dataframe result 
major_difference["ml_generate_text_llm_result"].iloc[0]

'The most obvious difference between the two lists of comments is that the comments in list 1 are written in a more informal and personal style, while the comments in list 2 are written in a more formal and legalistic style.'

We now see GeminiTextGenerator's characterization of the different comment groups. Thanks for using BigQuery DataFrames!

# Summary and next steps

You've used the ML and LLM capabilities of BigQuery DataFrames to help analyze and understand a large dataset of unstructured feedback.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).